### Scrapping one page (Immovlan)

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

from fake_useragent import UserAgent

In [2]:
# Url of website
url = "https://immovlan.be/en/detail/residence/for-sale/6800/libramont-chevigny/vbc87210"


In [3]:
ua = UserAgent()

In [8]:
url = "https://immovlan.be/en/detail/residence/for-sale/6800/libramont-chevigny/vbc87210"

ua = UserAgent()
headers = {
    "User-Agent": ua.random,
}

# set headers
response = requests.get(url, headers=headers)

content =response.content
print(response)

<Response [200]>


In [ ]:
the_soup = BeautifulSoup(content, 'html.parser')
the_soup

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0" name="viewport"/><meta content="app-id=968691127" name="apple-itunes-app"/><meta content="app-id=com.czaam" name="google-play-app"/><meta content="735734281535384549ac78e4353b4147" name="p:domain_verify"/><meta content="Residence for sale | Sur le Chene 30 - 6800 Libramont-Chevigny | 449 000 € | Livable surface 273m² | EPC D | 5 Bedrooms | 2 Bathrooms | Discover all pictures and details." name="description"/><meta content="House,Residence,for sale,Libramont-Chevigny,6800" name="keywords"/><meta content="noindex" name="robots"/><link href="https://immovlan.be/en/detail/residence/for-sale/6800/libramont-chevigny/vbc87210" rel="canonical"/><link href="https://immovlan.be/nl/detail/huis/te-koop/6800/libramont-chevigny/vbc87210" hreflang="nl" rel="alternate"/><link href="https://immovlan.be/fr/detail/maison/a-vendre/6800/libramont-chevigny/vbc87210" hreflang=

##### Get characteristics for property

In [63]:
property_characteristics = dict()

In [64]:
code = the_soup.find(class_='vlancode')
property_characteristics['property_code'] = code.text

In [65]:
type_of_property = the_soup.find(class_= 'detail__header_title_main')
property_characteristics['type_of_property'] = type_of_property.text.split()[0]

In [66]:
price = the_soup.find(class_='detail__header_price_data')
property_characteristics['price'] = price.text

In [67]:
locality = the_soup.find(class_='city-line')
property_characteristics['locality'] = locality.text

In [68]:
for tag in the_soup.find_all('h4', class_ =False):
   
    characteristic_name = "_".join(list(map(str.lower, tag.text.split())))
    print(characteristic_name)
    property_characteristics[characteristic_name] = tag.find_next().text

state_of_the_property
currently_leased
build_year
availability
number_of_bedrooms
surface_bedroom_1
surface_bedroom_2
surface_bedroom_3
surface_bedroom_4
surface_bedroom_5
livable_surface
furnished
surface_of_living-room
cellar
diningroom
surface_of_the_diningroom
garage
number_of_garages
kitchen_equipment
kitchen_type
surface_kitchen
number_of_bathrooms
number_of_toilets
surface_of_the_bathroom(s)
type_of_heating
type_of_glazing
elevator
access_for_disabled
orientation_of_the_front_facade
number_of_facades
frontage_width
number_of_parking_places_(outdoor)
garden
surface_garden
terrace
surface_terrace
terrace_orientation
total_land_surface
terrain_width_at_the_roadside
sewer_connection
running_water
specific_primary_energy_consumption
yearly_total_primary_energy_consumption
validity_date_epc/peb
co2_emission
certification_-_electrical_installation
building_permission_granted
planning_permission_granted
preemption_right
flooding_area_type
demarcated_flooding_area


In [69]:
for k,v in property_characteristics.items():
    print(f"{k}: {v}")

property_code: VBC87210
type_of_property: Residence
price:   449 000 €  
locality: 6800 Libramont-Chevigny
state_of_the_property: Normal
currently_leased: No
build_year: 1978
availability: Immediately
number_of_bedrooms: 5
surface_bedroom_1: 12 m²
surface_bedroom_2: 16 m²
surface_bedroom_3: 16 m²
surface_bedroom_4: 26 m²
surface_bedroom_5: 9 m²
livable_surface: 273 m²
furnished: No
surface_of_living-room: 26 m²
cellar: Yes
diningroom: Yes
surface_of_the_diningroom: 10 m²
garage: Yes
number_of_garages: 2
kitchen_equipment: Fully equipped
kitchen_type: Western kitchen
surface_kitchen: 16 m²
number_of_bathrooms: 2
number_of_toilets: 2
surface_of_the_bathroom(s): 9 m²
type_of_heating: Fuel oil
type_of_glazing: Double glass
elevator: No
access_for_disabled: No
orientation_of_the_front_facade: South-east
number_of_facades: 4
frontage_width: 16 m
number_of_parking_places_(outdoor): 3
garden: Yes
surface_garden: 780 m²
terrace: Yes
surface_terrace: 35 m²
terrace_orientation: North-west
total_l